[Ранее](/courses/recommendation-systems/factorization-machines-pytorch) мы разобрали алгоритм факторизационных машин, а именно, поняли, чем как переформулировать задачу заполнения пропусков в матрице в задачу регрессии, как при помощи регресии моделировать взаимодействие признаков и почему это едва ли возможно сделать в задаче рекоммендации обычным при помощи попарного произведения всех возможных признаков. Факторизационные машины решают проблему сложности моделирования взаимодействия между всеми парами признаков, используя некоторого рода уловку — они ставят в соответствие к каждому признаку вектор низкой размерности, координаты которого свидетельствуют о близости признаков между собой. Благодаря этому, а также формуле, выведенной автором из определения факторизационных машин, нам не нужно подбирать веса для всех возможных пар признаков, а достаточно определить координаты этих векторов, что делается за линейное время.



In [1]:
import torch
import torch.nn as nn
from sklearn.metrics import r2_score

from fastFM.datasets import make_user_item_regression
from sklearn.model_selection import train_test_split

X, y, _ = make_user_item_regression(n_user=100, n_item=50)
X_train, X_test, y_train, y_test = train_test_split(X, y)


class FieldAwareFactorizationMachine(nn.Module):
    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(sum(field_dims), embed_dim) for _ in range(self.num_fields)
        ])
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        for embedding in self.embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        xs = [self.embeddings[i](x) for i in range(self.num_fields)]
        ix = list()
        for i in range(self.num_fields - 1):
            for j in range(i + 1, self.num_fields):
                ix.append(xs[j][:, i] * xs[i][:, j])
        ix = torch.stack(ix, dim=1)
        return ix

In [13]:
X.toarray().shape

(5000, 150)

In [15]:
features_by_field = {
    "users": list(range(100)),
    "items": list(range(100, 150))
}

In [6]:
class FFM(nn.Module):
    def __init__(self, fields: dict, embed_dim=3):
        super().__init__()
        self.fields = fields
        self.embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(sum(field_dim), embed_dim) for field_name, field_dim in self.fields.items()
        ])
        for embedding in self.embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)
#         self.V = nn.Parameter(torch.randn(features_num, k), requires_grad=True)
        torch.nn.init.xavier_uniform_(self.V.data)
        self.linear = nn.Linear(features_num, 1)

    def forward(self, X):
        out_1 = ((X @ self.V) ** 2).sum(1, keepdim=True)
        out_2 = ((X ** 2) @ (self.V ** 2)).sum(1, keepdim=True)

        out_interaction = (out_1 - out_2) / 2
        out_linear = self.linear(X)
        return out_interaction + out_linear

In [4]:
from torch import optim

X_train_tensor, y_train_tensor = torch.from_numpy(X_train.toarray()), torch.from_numpy(y_train.reshape(-1, 1))

model = FM(X_train.shape[1])
criterion = nn.MSELoss() 
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(500):
    optimizer.zero_grad()
    predictions = model(X_train_tensor.float())
    loss = criterion(predictions, y_train_tensor.float())
    # get gradients
    loss.backward()
    # update parameters
    optimizer.step()
    
X_test_tensor = torch.from_numpy(X_test.toarray())
predictions = model(X_test_tensor.float())
r2_score(y_test, predictions.squeeze().detach().numpy())

0.9788741139133923

In [ ]:
1

http://ailab.criteo.com/ctr-prediction-linear-model-field-aware-factorization-machines/